**Importy**

In [45]:
import regex
import json, ast
from csv import writer
import pandas as pd 
import numpy as np
from stop_words import get_stop_words

In [46]:
import majka
from majka import Majka
morph = Majka('../data/w-lt.sk.fsa')

morph.flags |= majka.ADD_DIACRITICS  # find word forms with diacritics
morph.flags |= majka.DISALLOW_LOWERCASE  # do not enable to find lowercase variants
morph.flags |= majka.IGNORE_CASE  # ignore the word case whatsoever
morph.flags = 0  # unset all flags

morph.tags = False  # return just the lemma, do not process the tags
# morph.tags = True  # turn tag processing back on (default)

morph.first_only = True  # return only the first entry
# morph.first_only = False  # return all entries (default)

In [47]:
from corpy.morphodita import Tagger
tagger = Tagger("../data/slovak-morfflex-pdt-170914.tagger")

INFO:corpy.morphodita:Loading tagger.


In [48]:
import nltk
nltk.download('punkt')
# nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rusna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Premenne**

In [49]:
# file s jednou page
debug_filepath = "../data/test_data_1page.xml"
# cely file
input_filepath = "../data/skwiki-latest-pages-articles.xml"
output_filepath = "../data/anchors.csv"
THRESHOLD = 2

**Funkcie**

In [50]:
# otvorenie suboru, zadefinovanie hlavicky csv suboru a nasledne citanie s pracovanie po riadkoch
def open_read(input_filepath,output_filepath,process):
    with open(input_filepath, encoding='UTF-8') as input_file ,open(output_filepath, 'w+', encoding='UTF-8' ) as output_file:
        csv_writer = writer(output_file, delimiter='|')
        csv_writer.writerow(["Link", "Alt_Text"])
        for line in input_file:
            process(line,csv_writer)

In [81]:
# využitie regexu na nájdene všetkých zhôd v riadku a zapísanie do csv súboru. 
def process(line,csv_writer):
    regex_patern = '\[\[([^|\]\]]+)\|?([^|\]\]]+)?\]\]'
    #\[\[([^|\]\]]+)\|?([^|\]\]]+)?\]\]   #\[\[([\p{L}:\s\{\}\.\/\(\)]+)\|?([^\]]*)?\]\]
    matches = regex.findall(regex_patern, line) 
    for match in matches:
        list_matche = list(filter(None, match))
        
        
        csv_writer.writerow(list_matche)

In [52]:
# funkcia pre upravu textu
def replace(text, regex_pattern, substitution):
    if not isinstance(text, str):
        return text
    return regex.sub(regex_pattern,substitution,text)

In [53]:
# funkcia pre tokenizovanie textu. Text -> Vety -> Slova (Tokeny) vyuzitim nltk 
def tokenizer_modded(text):
    sentences = nltk.sent_tokenize(text)
    tokens = []
    for sent in sentences:
        tokens.append(nltk.word_tokenize(sent))
    flatten_list = [j for sub in tokens for j in sub] 
    result = [token.lower() for token in flatten_list if token not in ".,?!...-"]
    
    return result

In [72]:
# lematizovanie textu s odstranenim stop slov. Vyuziva sa majka so Slovenskym Slovnikom
def lematizer_modded(text):
    stop_words = get_stop_words('sk')

    result = []
    for word in ast.literal_eval(text):
        morph_word = morph.find(word)
#         Aktualne slova ktore nemaju lemu, budu mat povodny tvar.
        if morph_word:
            result.append(morph_word[0]['lemma'])
        else:
            if hashmap_unlem[word] > THRESHOLD:
                result.append(word)
            
    for word in result:  # odstranenie stopslov
        if word in stop_words:
            result.remove(word)
    return result

In [73]:
def fill_hashmap(text):
  
    for word in text:
        morph_word = morph.find(word)
        if morph_word == []:
            
            if not word in hashmap_unlem:
                hashmap_unlem[word] = 1
            else:
                hashmap_unlem[word] += 1 

    

In [85]:
def process_data(df):
    for column in df:
        df[column] = df[column] \
        .apply(lambda x: x if x is not np.NaN else None) \
        .apply(replace,regex_pattern = r"[\\\/\(\):#.\_?!]|({{.*}})|\d*px\d*png\d*svg\d*jpg",substitution =' ')\
        .apply(replace,regex_pattern = r"([ ]+)",substitution =' ')\
        
    return df

In [75]:
def tokenize_data(df):
    for column in df:
        df[column] = df[column] \
        .apply(lambda text: tokenizer_modded(text) if text is not None else [] )
        
    return df

In [76]:
def lemetize_data(df):
    for column in df:
        df[column] = df[column] \
        .apply(lambda text: lematizer_modded(text) )
        
    return df

In [77]:
def create_hashmap(df):
    for column in df:
#         print(column)
        df[column] \
        .apply(lambda text: fill_hashmap(text) )
        


**Priebeh spracovania**

In [86]:
open_read(debug_filepath,output_filepath,process)

In [87]:
# spracovanie suboru po chunkoch a postupne ukladanie vysledkov do csv
hashmap_unlem = {}  
for index,chunk in enumerate(pd.read_csv(output_filepath, chunksize=15, delimiter='|')):
    

    process_data(chunk)
    chunk = tokenize_data(chunk)
    create_hashmap(chunk)
    chunk.to_csv('../data/tokenized_data.csv', mode = 'a+', index=False, header=False, encoding='UTF-8')




In [88]:
for index,chunk in enumerate(pd.read_csv('../data/tokenized_data.csv',chunksize=15, names=["Link","Alt_Text"])):
    

    lemetize_data(chunk)
    
    print('chunk ------------------------------------------------------------------------------',index)    
    print(chunk)
    chunk.to_csv('../data/done.csv', mode = 'a+', index=False, encoding='UTF-8')

chunk ------------------------------------------------------------------------------ 0
                                                 Link  \
0                                         [slovensko]   
1                              [slovenský, republika]   
2                                         [slovensko]   
3                                      [generálmajor]   
4                                  [ľubomír, svoboda]   
5   [dopravné, krídlo, generál, milana, rastislava...   
6   [taktický, krídlo, generálmajor, otta, smika, ...   
7   [vrtuľníkový, krídlo, generálplukovník, jána, ...   
8         [protilietadlový, raketový, brigáda, nitra]   
9      [brigáda, riadenie, vzdušný, operácia, zvolen]   
10                                            [súbor]   
11            [ozbrojený, sila, slovenský, republika]   
12                         [súbor, of, slovakia, svg]   
13                             [súbor, slovakia, svg]   
14                             [aero, l-39, albatros]   



chunk ------------------------------------------------------------------------------ 21
                                               Link  \
315                  [sikorsky, uh-60, black, hawk]   
316                                    [mil, mi-17]   
317                                    [mil, mi-17]   
318     [protilietadlový, raketový, brigáda, nitra]   
319                                         [nitra]   
320                                         [s-300]   
321                                     [2k12, kub]   
322                                          [9k38]   
323  [brigáda, riadenie, vzdušný, operácia, zvolen]   
324                                        [zvolen]   
325                                         [radar]   
326                             [stíhací, lietadlo]   
327                       [mikojan-gurevič, mig-29]   
328      [súbor, slovak, air, force, mig-29as, jpg]   
329                                         [rusko]   

                               

599            [mikojan-gurevič, mig-29]        [mig-29]
chunk ------------------------------------------------------------------------------ 40
                                                  Link  \
600                                        [1, január]   
601                                                 []   
602  [veliteľstvo, vzdušný, sila, ozbrojený, sila, ...   
603                                                 []   
604                                              [jún]   
605                                  [biely, albatros]   
606                             [aero, l-39, albatros]   
607                                                 []   
608                                        [2k12, kub]   
609                                         [november]   
610                              [stredoeurópsky, čas]   
611            [organizácia, severoatlantický, zmluva]   
612                                                 []   
613                      [suchoj, su-17, su

chunk ------------------------------------------------------------------------------ 58
                                      Link            Alt_Text
870                              [tupolev]                  []
871    [súbor, slovak, tupolev, aero, jpg]             [150px]
872                      [sovietsky, zväz]              [zssr]
873                     [cvičný, lietadlo]  [cvičný, lietadlo]
874                   [aero, l-29, delfín]                  []
875                     [súbor, l-29, jpg]             [150px]
876                      [česko-slovensko]                  []
877                          [helikoptéra]          [vrtuľník]
878                            [mil, mi-2]                  []
879           [súbor, mi-2, slovakia, jpg]             [150px]
880                      [sovietsky, zväz]              [zssr]
881                               [poľsko]                  []
882                            [mil, mi-8]                  []
883  [súbor, slovak, air, forc

In [89]:
# TODO
# - zmenit regex na vseobecny
# - analyza
